In [2]:
!pip install textattack

     |████████████████████████████████| 215kB 7.0MB/s 
     |████████████████████████████████| 778kB 16.8MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 1.4MB 37.8MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 317kB 38.3MB/s 
     |████████████████████████████████| 1.7MB 40.2MB/s 
     |████████████████████████████████| 3.0MB 39.9MB/s 
     |████████████████████████████████| 686kB 42.8MB/s 
     |████████████████████████████████| 778kB 38.0MB/s 
     |████████████████████████████████| 245kB 44.6MB/s 
     |████████████████████████████████| 122kB 44.3MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 163kB 42.1MB/s 
     |████████████████████████████████| 102kB 12.3MB/s 
     |████████████████████████████████| 245kB 40.5MB/s 
     |████████████████████████████████| 17.3MB 216kB/s 
     |████████████████████████████████| 204kB 41.1MB/

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
df=pd.read_csv('/content/drive/My Drive/combined_relevant_data.csv')

In [6]:
df['text'][0]

'The decision to remove Chick-fil-A from Emory University was based solely on student feedback.'

In [7]:
dataset=[]
for i in range(len(df)):
  label=df['truth'][i]
  text_input=df['text'][i]
  if label == 1:
    dataset.append((text_input,1))
  else:
      dataset.append((text_input,0))

In [8]:
dataset[2]

('Ron Klein repeatedly voted in favor of amnesty for illegal immigrants.', 0)

In [10]:
from textattack.attack_recipes import BERTAttackLi2020


In [11]:
import textattack

In [12]:
import torch

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 7.3MB/s 


In [14]:
from pytorch_pretrained_bert import BertTokenizer

In [15]:
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [16]:
from textattack.models.wrappers import HuggingFaceModelWrapper

In [17]:
model = torch.load('/content/drive/My Drive/bert_model_128_128_8.zip (Unzipped Files)/bert_model_128_128_8.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or

In [ ]:
print(model)

In [18]:
def preprocess_for_BERT(sentences, tokenizer, MAX_LEN):
        # sentences = df['text'].values
        sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
        input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
        input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
        
        attention_masks = []
        for seq in input_ids:
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        return input_ids, attention_masks  

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(n_gpu, torch.cuda.get_device_name(0))

1 Tesla K80


In [20]:
class CustomBFSQModelWrapper(HuggingFaceModelWrapper):
  def __init__(self, model, tokenizer):
    self.model = model
    self.model.eval()
    self.model.cuda()
    self.MAX_LEN=128
    self.tokenizer = tokenizer
  
  def __call__(self, text_input_list):
    input_ids, attention_masks = preprocess_for_BERT(text_input_list, self.tokenizer, self.MAX_LEN)
    predictions=[]
    input_ids=torch.cuda.LongTensor(input_ids)
    attention_masks=torch.cuda.LongTensor(attention_masks)
    # input_ids.to(device)
    # attention_masks.to(device)
    with torch.no_grad():
      logits=self.model(input_ids, token_type_ids=None, attention_mask=attention_masks)
    logits = logits.detach().cpu()
    # logits=logits.unbind()
    # for i in range(len(logits)):
    #   predictions.append(logits[i])
    # predictions=torch.Tensor(predictions)
    return logits

    
    
  

In [21]:
wrapper=CustomBFSQModelWrapper(model, tokenizer)

In [22]:
wrapper.__call__(['Donald Trump is the worst president in United States history.','President Obama was born in Kenya and is a Muslim'])

tensor([[-0.2581,  1.6357],
        [ 1.9370, -1.1452]])

In [23]:
attack = BERTAttackLi2020.build(wrapper)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentenc

In [24]:
len(dataset)

47503

In [ ]:
from textattack.loggers import CSVLogger
logger=CSVLogger()

In [ ]:
results=[]
results_iterable = attack.attack_dataset(dataset, indices=range(len(dataset)))
for result in results_iterable:
    logger.log_attack_result(result)
    print(result.__str__(color_method='ansi'))

1 (100%) --> 0 (61%)

The decision to remove Chick-fil-A from Emory University was based solely on student feedback.

The deciding to remove Chick-fil-A from Emory University was based solely on performance feedback.
1 (100%) --> 0 (81%)

Mike Dovillas big ideas like expanding the Ohio Motion Picture Tax Credit, which helped bring films like The Avengers to Cleveland, have helped make our state No. 1 in the Midwest for job creation.

bill Dovillas big ideas like expanding the Ohio Motion Picture subsidy Credit, which helped bring films like The Avengers to Cleveland, have left make our state No. 1 in the Midwest for job creation.
1 (96%) --> [SKIPPED]

Ron Klein repeatedly voted in favor of amnesty for illegal immigrants.
1 (100%) --> 0 (98%)

Since the salary for state lawmakers was raised to $79,500, that purchasing power has now been reduced to $53,000.

on the salary for state laws was raised to $79,500, that purchasing power has now been reduced to $53,000.
0 (99%) --> 1 (99%)

Be

In [ ]:
logger.flush()

In [1]:
from textattack.loggers import CSVLogger
logger=CSVLogger()

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
logger=CSVLogger()

In [32]:
for result in results_iterable:
  logger.log_attack_result(result)


In [36]:
logger.flush()

In [37]:
dfResults=pd.read_csv('results.csv')

In [45]:
len(dfResults)

10